In [3]:
import numpy as np
from qiskit import QuantumCircuit
import torch
import matplotlib as plt
import pdflatex
import pylatexenc
import gym
import torch

# Quick Check to see if my GPU is being detected
device = torch.device("cuda" if torch.cuda.is_available() else  "cpu")
print("Using ", device)

Using  cpu
